In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing Lib

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk 
nltk.download('stopwords')

print(len(stopwords.words('english')))
print(stopwords.words('english')[:20])

# Loading data

In [ ]:
train_data = pd.read_csv('/kaggle/input/fake-news/train.csv')
train_data.shape

In [ ]:
test_data = pd.read_csv('/kaggle/input/fake-news/test.csv')
test_data.shape

In [ ]:
print(test_data.isna().sum())
test_data.dropna(inplace=True)
id_ = test_data.id


In [ ]:
train_data.head()

# Pre-Processing 

In [ ]:
train_data.isna().sum()

In [ ]:
train_data.dropna(inplace=True)
train_data.shape

In [ ]:
y = train_data['label'].values
train_data.drop(['label'] ,axis=1, inplace=True)

In [ ]:
train_data.head()

In [ ]:
X = pd.concat([train_data ,test_data] ) 


In [ ]:
print(X.shape)
print(y.shape)
X['content']=X['author']+" "+X['title']
X.head()

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def normalize(content):
    content_ = re.sub('[^a-zA-Z]',' ',content)
    content_ = content_.lower()
    content_ = content_.split()
    content_ = [stemmer.stem(word) for word in content_ if not word in stopwords.words('english')]
    content_ = ' '.join(content_)
    return content_


In [ ]:

X['content'] = X['content'].apply(normalize)
X

In [ ]:
def tfIDF(data):
    return TfidfVectorizer().fit_transform(data)
    

In [ ]:
tf = tfIDF(X['content'].values).toarray()
X.head()

In [ ]:

tf_train = tf[:train_data.shape[0],:]
print(tf_train.shape)
tf_test = tf[train_data.shape[0] :,:]
print(tf_test.shape)

In [ ]:
len(y)

In [ ]:
x_train ,x_test, y_train ,y_test = train_test_split(tf_train,y , test_size=0.2)

In [ ]:
model_Log = LogisticRegression()
model_Log.fit(x_train, y_train)

In [ ]:
# accutrcy in train 

x_train_prediction = model_Log.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

In [ ]:
# accutrcy in test
X_test_prediction = model_Log.predict(x_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

In [ ]:
y_pred = model_Log.predict(tf_test)
y_pred


In [ ]:
# df = pd.DataFrame([id_,y_pred ],columns=['label','id'] )
# df = pd.DataFrame({'label': y_pred.values, 'id': id_.values})
Id = id_.values

In [ ]:
print(type(y_pred))
print(type(Id))



In [ ]:
df = pd.DataFrame({'id': Id , 'label': y_pred})

In [ ]:
df.head()

In [ ]:
df.to_csv('submit.csv', index=False) 